In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as bs
import mysql.connector
import time

In [ ]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="jobs-scraping.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user="root",
    password="123456789",
    database="job_scraping_database"
)

cursor = mydb.cursor()

# # Create the table if it doesn't exist
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS Marketing_Jobs_Indeed (
#         JobId Varchar(20) PRIMARY KEY,
#         Title VARCHAR(200),
#         Company VARCHAR(100),
#         Location VARCHAR(100),
#         Apply VARCHAR(300),
#         JobDescription TEXT,
#         Salary VARCHAR(100)
#     )
# ''')

In [ ]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")

In [ ]:
wd = webdriver.Chrome(options = chrome_options)
time.sleep(1)

driver = webdriver.Chrome(options = chrome_options)
time.sleep(1)

In [ ]:
def indeed_scraper(webpage, page_number):
    next_page = webpage + str(page_number)
    print(str(next_page))
    wd.get(next_page)
    
    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_ = "job_seen_beacon")
    
    for job in jobs:
        job_head_elem = job.find("h2", class_ = "jobTitle")
        #JobId
        job_id = job_head_elem.find("span")["id"].split("-")[1]
        
        #Check if the JobId already exists in the table
        select_query = "SELECT JobId FROM marketing_jobs WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()
        
        if result is None:
            #Job Title
            job_title = job_head_elem.find("span").text.strip()
            
            #Job Link
            job_link = "https://www.indeed.com" + job_head_elem.find("a")["href"]
            
            #Company Name
            company_name = job.find("span", class_ = "companyName").text.strip()
            
            #Location
            location = job.find("div", class_ = "companyLocation").text.strip()
            
            driver.get(job_link)
            time.sleep(3)
            job_src = driver.page_source
            job_soup = bs(job_src, "html.parser")
            
            #Salary
            salary_elem = None
            for elem in job_soup.find_all("div", class_ = "css-tvvxwd ecydgvn1"):
                text = elem.text.strip()
                if any(char.isdigit() for char in text):
                    salary_elem = elem
                    break
            salary = salary_elem.text.strip() if salary_elem else ""
            
            #Job Description
            job_des_elem = job_soup.find("div", id = "jobDescriptionText")
            job_des = job_des_elem.text.strip() if job_des_elem else ""
            
            #Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO marketing_jobs (JobId, Title, Company, Location, Apply, Salary, JobDescription)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
        else:
            print("Skipping duplicate JobId:", job_id)
    print("Data updated")
    
    if page_number < 1000:
        page_number += 10
        indeed_scraper(webpage, page_number)
    else:
        cursor.close()
        mydb.close()
        driver.quit
        wd.quit
        
indeed_scraper('https://www.indeed.com/jobs?q=marketing&l=United+States&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&radius=35&start=', 0)
            